# 3. Working with relational databases in Python

Dalam bab ini, Anda akan belajar cara mengekstrak data yang bermakna dari **relational databases**, keterampilan penting bagi ilmuwan data mana pun. Anda akan belajar tentang model relasional, cara membuat kueri SQL, cara memfilter dan membuat perintah SQL records Anda, dan bagaimana melakukan kueri lanjutan dengan menggabungkan tabel database.

## Introduction to relational databases

### The relational model

Manakah dari berikut ini yang bukan bagian dari model relasional?

**BUKAN BAGIAN MODEL RELASIONAL**
* Database terdiri dari setidaknya 3 tabel.

**BAGIAN MODEL RELASIONAL**
* Setiap baris atau catatan dalam tabel mewakili turunan dari tipe entitas.
* Setiap kolom dalam tabel mewakili atribut atau fitur dari instance.
* Setiap tabel berisi kolom *primary key*, yang memiliki entri unik untuk setiap baris.
* Ada hubungan antar tabel.

## Creating a database engine in Python

### Creating a database engine

Di sini, Anda akan menjalankan SQL engine pertama Anda. Anda akan membuat engine untuk terhubung ke database SQLite `'Chinook.sqlite'`, yang ada di direktori kerja Anda. Ingat bahwa untuk membuat engine yang terhubung ke `'Northwind.sqlite'`, Hugo menjalankan perintah:

<pre>engine = create_engine('sqlite:///Northwind.sqlite')</pre>

Di sini, `'sqlite:///Northwind.sqlite'` disebut *connection string* ke database SQLite `Northwind.sqlite`. Sedikit latar belakang pada [database Chinook](https://github.com/lerocha/chinook-database): database Chinook berisi informasi tentang penyimpanan media digital semi-fiksi di mana data media nyata dan data pelanggan, karyawan, dan penjualan dibuat secara manual.

Mengapa nama Chinook, Anda bertanya? Menurut [situs web](https://github.com/lerocha/chinook-database) mereka,

*Nama database sampel ini didasarkan pada database Northwind. Chinook adalah angin di pedalaman Barat Amerika Utara, di mana Prairi Kanada dan Great Plains bertemu berbagai pegunungan. Chinook paling banyak ditemukan di Alberta selatan di Kanada. Chinook adalah pilihan nama yang bagus untuk database yang bermaksud menjadi alternatif bagi Northwind.*

In [1]:
# Import necessary module
from sqlalchemy import create_engine

# Create engine: engine
engine = create_engine('sqlite:///Chinook.sqlite')

### What are the tables in the database?

Dalam latihan ini, Anda akan sekali lagi membuat engine untuk terhubung ke `'Chinook.sqlite'`. Namun, sebelum Anda bisa mendapatkan data apa pun dari database, Anda harus tahu tabel apa yang di dalamnya!

Untuk tujuan ini, Anda akan menyimpan nama tabel ke `list` menggunakan metode `table_names()` pada engine dan kemudian Anda akan mencetak list.

In [2]:
# Import necessary module
from sqlalchemy import create_engine

# Create engine: engine
engine = create_engine('sqlite:///Chinook.sqlite')

# Save the table names to a list: table_names
table_names = engine.table_names()

# Print the table names to the shell
print(table_names)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Querying relational databases in Python

### The Hello World of SQL Queries!

Dalam latihan ini, Anda akan berkenalan dengan *Hello World* dari kueri SQL yaitu `SELECT`, untuk mengambil semua kolom dari tabel `Album` di database `Chinook`. Ingat bahwa kueri `SELECT *` memilih semua kolom.

In [3]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
engine = create_engine('sqlite:///Chinook.sqlite')

# Open engine connection: con
con = engine.connect()

# Perform query: rs
rs = con.execute('SELECT * FROM Album')

# Save results of the query to DataFrame: df
df = pd.DataFrame(rs.fetchall())

# Close connection
con.close()

# Print head of DataFrame df
print(df.head())

   0                                      1  2
0  1  For Those About To Rock We Salute You  1
1  2                      Balls to the Wall  2
2  3                      Restless and Wild  2
3  4                      Let There Be Rock  1
4  5                               Big Ones  3


### Customizing the Hello World of SQL Queries

Selamat telah mengeksekusi kueri SQL pertama Anda! Sekarang Anda akan mengetahui cara menyesuaikan kueri Anda untuk:

* Pilih kolom tertentu dari tabel;
* Pilih jumlah baris tertentu;
* Import nama kolom dari tabel database.

Ingat bahwa Hugo melakukan kustomisasi kueri yang sangat mirip dalam video:

<pre>
engine = create_engine('sqlite:///Northwind.sqlite')

with engine.connect() as con:
    rs = con.execute("SELECT OrderID, OrderDate, ShipName FROM Orders")
    df = pd.DataFrame(rs.fetchmany(size=5))
    df.columns = rs.keys()
</pre>

Paket telah diimpor sebagai berikut:

<pre>
from sqlalchemy import create_engine
import pandas as pd
</pre>

Engine juga sudah dibuat:

<pre>engine = create_engine('sqlite:///Chinook.sqlite')</pre>

Koneksi engine sudah terbuka dengan pernyataan:

<pre>with engine.connect() as con:</pre>

Semua kode yang Anda perlu lengkapi berada dalam konteks ini.

In [4]:
# Open engine in context manager
# Perform query and save results to DataFrame: df
with engine.connect() as con:
    rs = con.execute("SELECT LastName, Title FROM Employee")
    df = pd.DataFrame(rs.fetchmany(size=3))
    df.columns = rs.keys()

# Print the length of the DataFrame df
print(len(df))

# Print the head of the DataFrame df
print(df.head())

3
  LastName                Title
0    Adams      General Manager
1  Edwards        Sales Manager
2  Peacock  Sales Support Agent


### Filtering your database records using SQL's WHERE

Sekarang Anda dapat menjalankan kueri SQL dasar untuk memilih catatan dari tabel apa pun di database Anda dan Anda juga dapat melakukan kustomisasi kueri sederhana untuk memilih kolom dan jumlah baris tertentu.

Ada beberapa serangkaian kueri SQL standar yang akan membantu Anda dalam perjalanan Anda untuk menjadi SQL ninja.

Misalnya, Anda ingin mendapatkan semua catatan dari tabel `Customer` dari database Chinook yang dimana `Country` adalah `'Canada'`. Anda dapat melakukan ini dengan sangat mudah dalam SQL menggunakan pernyataan `SELECT` diikuti oleh klausa `WHERE` sebagai berikut:

<pre>SELECT * FROM Customer WHERE Country = 'Canada'</pre>

Bahkan, Anda bisa memfilter pernyataan `SELECT` apa pun dengan kondisi apa pun menggunakan klausa `WHERE`. Ini disebut *memfilter* catatan Anda.

Dalam latihan interaktif ini, Anda akan memilih semua catatan dari tabel `Employee` di mana `'EmployeeId'` lebih besar atau sama dengan `6`.

In [6]:
# Create engine: engine
engine = create_engine('sqlite:///Chinook.sqlite')

# Open engine in context manager
# Perform query and save results to DataFrame: df
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Employee WHERE EmployeeId >= 6")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

# Print the head of the DataFrame df
df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
1,7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
2,8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


### Ordering your SQL records with ORDER BY

Anda juga dapat membuat perintah urutan (*Order*) hasil kueri SQL Anda. Misalnya, jika Anda ingin mendapatkan semua catatan dari tabel `Customer` dari database `Chinook` dan membuat perintah dalam urutan *meningkat* dengan kolom `SupportRepId`, Anda bisa melakukannya dengan kueri berikut:

<pre>"SELECT * FROM Customer ORDER BY SupportRepId"</pre>

Bahkan, Anda dapat mengurutkan pernyataan `SELECT` dengan kolom apa pun.

Dalam latihan interaktif ini, Anda akan memilih semua catatan dari tabel `Customer` dan membuat perintah urutan *meningkat* pada kolom `BirthDate`.

In [7]:
# Create engine: engine
engine = create_engine("sqlite:///Chinook.sqlite")

# Open engine in context manager
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Employee ORDER BY BirthDate")
    df = pd.DataFrame(rs.fetchall())

    # Set the DataFrame's column names
    df.columns = rs.keys()

# Print head of DataFrame
df.head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
3,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
4,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


## Querying relational databases directly with pandas

### Pandas and The Hello World of SQL Queries!

Di sini, Anda akan memanfaatkan kekuatan `pandas` untuk menulis hasil kueri SQL Anda ke DataFrame dalam satu baris cepat kode Python!

Anda pertama-tama akan mengimpor `pandas` dan membuat mesin SQLite `'Chinook.sqlite'`. Kemudian Anda akan meminta database untuk memilih semua catatan dari tabel `Album`.

Ingatlah bahwa untuk memilih semua catatan dari tabel `Orders` di database `Northwind`, Hugo menjalankan perintah berikut:

<pre>df = pd.read_sql_query("SELECT * FROM Orders", engine)</pre>

In [8]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
engine = create_engine("sqlite:///Chinook.sqlite")

# Execute query and store records in DataFrame: df
df = pd.read_sql_query("SELECT * FROM Album", engine)

# Print head of DataFrame
print(df.head())

# Open engine in context manager and store query result in df1
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Album")
    df1 = pd.DataFrame(rs.fetchall())
    df1.columns = rs.keys()

# Confirm that both methods yield the same result
print(df.equals(df1))

   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3
True


### Pandas for more complex querying

Di sini, Anda akan menjadi lebih akrab dengan fungsi pandas `read_sql_query()` dengan menggunakannya untuk menjalankan kueri yang lebih kompleks: pernyataan `SELECT` diikuti oleh klausa `WHERE` DAN klausa `ORDER BY`.

Anda akan membangun DataFrame yang berisi baris-baris dari tabel `Employee` di mana `EmployeeId` lebih besar dari atau sama dengan `6` dan Anda akan membuat urutan entri-entri ini dengan `BirthDate`.

In [9]:
# Import packages
from sqlalchemy import create_engine
import pandas as pd

# Create engine: engine
engine = create_engine("sqlite:///Chinook.sqlite")

# Execute query and store records in DataFrame: df
df = pd.read_sql_query("SELECT * FROM Employee WHERE EmployeeId >= 6 ORDER BY BirthDate", engine)

# Print head of DataFrame
print(df.head())

   EmployeeId  LastName FirstName       Title  ReportsTo            BirthDate  \
0           8  Callahan     Laura    IT Staff          6  1968-01-09 00:00:00   
1           7      King    Robert    IT Staff          6  1970-05-29 00:00:00   
2           6  Mitchell   Michael  IT Manager          1  1973-07-01 00:00:00   

              HireDate                      Address        City State Country  \
0  2004-03-04 00:00:00                  923 7 ST NW  Lethbridge    AB  Canada   
1  2004-01-02 00:00:00  590 Columbia Boulevard West  Lethbridge    AB  Canada   
2  2003-10-17 00:00:00         5827 Bowness Road NW     Calgary    AB  Canada   

  PostalCode              Phone                Fax                    Email  
0    T1H 1Y8  +1 (403) 467-3351  +1 (403) 467-8772    laura@chinookcorp.com  
1    T1K 5N8  +1 (403) 456-9986  +1 (403) 456-8485   robert@chinookcorp.com  
2    T3B 0C5  +1 (403) 246-9887  +1 (403) 246-9899  michael@chinookcorp.com  


## Advanced querying: exploiting table relationships

### The power of SQL lies in relationships between tables: INNER JOIN

Di sini, Anda akan melakukan `INNER JOIN` pertama Anda! Anda akan bekerja dengan database SQLite favorit Anda, `Chinook.sqlite`. Untuk setiap catatan dalam tabel `Album`, Anda akan mengekstrak `Title` bersama dengan `Name` Artis. Yang terakhir akan datang dari tabel `Artist` dan jadi Anda harus membuat `INNER JOIN` dengan kedua tabel ini di kolom `ArtistID` pada keduanya.

Ingat bahwa untuk `INNER JOIN` dengan tabel `Orders` dan `Customers` dari database Northwind, Hugo mengeksekusi query SQL berikut:

<pre>
"SELECT OrderID, CompanyName FROM Orders INNER JOIN Customers on Orders.CustomerID = Customers.CustomerID"
</pre>

Kode berikut telah dijalankan untuk mengimpor paket yang diperlukan dan untuk membuat engine:

<pre>
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///Chinook.sqlite')
</pre>

In [10]:
# Open engine in context manager
# Perform query and save results to DataFrame: df
with engine.connect() as con:
    rs = con.execute("SELECT Title, Name FROM Album INNER JOIN Artist on Album.ArtistID = Artist.ArtistID")
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

# Print head of DataFrame df
print(df.head())

                                   Title       Name
0  For Those About To Rock We Salute You      AC/DC
1                      Balls to the Wall     Accept
2                      Restless and Wild     Accept
3                      Let There Be Rock      AC/DC
4                               Big Ones  Aerosmith


### Filtering your INNER JOIN

Selamat telah melakukan `INNER JOIN` pertama, Anda sekarang akan menyelesaikan bab ini dengan satu latihan terakhir di mana Anda melakukan `INNER JOIN` dan memfilter hasilnya menggunakan klausa `WHERE`.

In [12]:
# Execute query and store records in DataFrame: df
df = pd.read_sql_query("SELECT * FROM PlaylistTrack INNER JOIN Track on PlaylistTrack.TrackId = Track.TrackId WHERE Milliseconds < 250000", engine)

# Print head of DataFrame
df.head()

,PlaylistId,TrackId,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,3390,3390,One and the Same,271,2,23,None,217732,3559040,0.99
1,1,3392,3392,Until We Fall,271,2,23,None,230758,3766605,0.99
2,1,3393,3393,Original Fire,271,2,23,None,218916,3577821,0.99
3,1,3394,3394,Broken City,271,2,23,None,228366,3728955,0.99
4,1,3395,3395,Somedays,271,2,23,None,213831,3497176,0.99
